# Mini Project
## 3C - Auto detection

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2
import json
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

from sklearn.model_selection import train_test_split

import os
from tqdm import tqdm

### Create text file annotations for each image   

In [23]:
cd ..

/home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3


In [26]:
####### Converts a file filled with all image annotations to each annotation file for each image 

bbs = json.loads(open('./auto_det_chal_train_7oct/bbs/bbs.json').read())

train_size = 700 
val_size = 100

##################### Training Images ####################
for i in range(train_size):
    im = Image.open('./auto_det_chal_train_7oct/images/'+ str(i) + '.jpg')
    width, height = im.size #Image width and heights
    boxes = bbs[i]

    f = open('./auto_det_chal_train_7oct/data/labels/train/'+ str(i) + ".txt", "w")
    f.close()
    
    for j in range(len(boxes)):
        x, y = boxes[j][0]                  # Top left cordinate of the box
        h = abs(boxes[j][-1][1] - y)  # Height of the box
        w = abs(boxes[j][1][0] - x)   # Weight of the box
        if (x+w/2)/width < 0 or (x+w/2)/width > 1 or (y+h/2)/height < 0 or (y+h/2)/height > 1:
            os.remove('./auto_det_chal_train_7oct/data/images/train/' + str(i) + '.jpg')
            os.remove('./auto_det_chal_train_7oct/data/labels/train/' + str(i) + '.txt')
            break
        else:
            f = open('./auto_det_chal_train_7oct/data/labels/train/'+ str(i) + ".txt", "a")
            f.write(f"0 {(x+w/2)/width} {(y+h/2)/height} {w/width} {h/height}\n")
            f.close()

#################### Validation Images ####################
for i in range(train_size,val_size + train_size):
    im = Image.open('./auto_det_chal_train_7oct/images/'+ str(i) + '.jpg')
    width, height = im.size
    boxes = bbs[i]

    f = open('./auto_det_chal_train_7oct/data/labels/val/'+ str(i) + ".txt", "w")
    f.close()
    for j in range(len(boxes)):
        x, y = boxes[j][0]
        h = abs(boxes[j][-1][1] - y)
        w = abs(boxes[j][1][0] - x)
        if (x+w/2)/width < 0 or (x+w/2)/width > 1 or (y+h/2)/height < 0 or (y+h/2)/height > 1:
            os.remove('./auto_det_chal_train_7oct/data/images/val/' + str(i) + '.jpg')
            os.remove('./auto_det_chal_train_7oct/data/labels/val/' + str(i) + '.txt')
            break
        else:
            f = open('./auto_det_chal_train_7oct/data/labels/val/'+ str(i) + ".txt", "a")
            f.write(f"0 {(x+w/2)/width} {(y+h/2)/height} {w/width} {h/height}\n")
            f.close()

### Clone yolov5 algorithm from github and install requirements

In [13]:
# !git clone https://github.com/ultralytics/yolov5.git
# !pip install -r requirements.txt

### Change durrent directory to yolov5 directory to start training

In [3]:
%cd yolov5/

/home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/yolov5


### Training

In [4]:
# %pip install -q wandb
import wandb
wandb.login()

wandb: Currently logged in as: srinivas_123 (use `wandb login --relogin` to force relogin)


True

In [5]:
torch.cuda.empty_cache()

In [6]:
%timeit
!python3 train.py --img 640 --batch 4 --epochs 30 --workers 8 --cfg ./models/yolov5s.yaml --data ../auto_det_chal_train_7oct/data/auto.yaml --weights yolov5s.pt

wandb: Currently logged in as: srinivas_123 (use `wandb login --relogin` to force relogin)
train: weights=yolov5m.pt, cfg=./models/yolov5m.yaml, data=../auto_det_chal_train_7oct/data/auto.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 19 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/home/dhamodhar/Notes/Sem': No such file or directory
YOLOv5 🚀 2022-3-22 torch 1.8.0 CUDA:0 (NVIDIA GeForce GTX 1050, 4042MiB)

hyperparameter

### Plots

In [31]:
pwd

'/home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/yolov5'

### Testing on given dataset

In [35]:
%timeit
!python3 detect.py --img 640 --source ../auto_det_chal_train_7oct/data/images/test/ --weights ./runs/train/exp43/weights/best.pt --conf-thres 0.4

detect: weights=['./runs/train/exp43/weights/best.pt'], source=../auto_det_chal_train_7oct/data/images/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
fatal: cannot change to '/home/dhamodhar/Notes/Sem': No such file or directory
YOLOv5 🚀 2022-3-22 torch 1.8.0 CUDA:0 (NVIDIA GeForce GTX 1050, 4042MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (1).jpg: 640x480 Done. (0.023s)
image 2/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Dataclus

image 47/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (143).jpg: 640x480 3 autos, Done. (0.021s)
image 48/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (144).jpg: 640x480 2 autos, Done. (0.018s)
image 49/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (145).jpg: 640x480 Done. (0.018s)
image 50/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (146).jpg: 640x480 1 auto, Done. (0.019s)
image 51/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (147).jpg: 640x480 1 auto, Done. (0.021s)
image 52/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (148).jpg: 640x480 3 autos, Done. (0.018s)
image 53/152 /h

image 97/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (48).jpg: 640x480 Done. (0.018s)
image 98/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (49).jpg: 640x480 Done. (0.018s)
image 99/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (50).jpg: 640x480 1 auto, Done. (0.018s)
image 100/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (51).jpg: 640x480 1 auto, Done. (0.018s)
image 101/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (52).jpg: 640x480 Done. (0.020s)
image 102/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (53).jpg: 640x480 Done. (0.018s)
image 103/152 /home/dhamodhar/Notes/Sem 6/VR/

image 147/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (94).jpg: 640x480 1 auto, Done. (0.019s)
image 148/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (95).jpg: 640x480 Done. (0.018s)
image 149/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (96).jpg: 640x480 Done. (0.018s)
image 150/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (97).jpg: 640x480 1 auto, Done. (0.018s)
image 151/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (98).jpg: 640x480 1 auto, Done. (0.018s)
image 152/152 /home/dhamodhar/Notes/Sem 6/VR/Graded Assignment 3/auto_det_chal_train_7oct/data/images/test/Datacluster Auto (99).jpg: 640x480 1 auto, Done. (0.018s)
Speed: 0.6ms pre-process, 